In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import sys

sys.path.append('..')
import gryds
import scipy
plt.rcParams['image.cmap'] = 'gray'

# Gryds: a Python package for geometric transformations for augmentations in deep learning

This package enables you to make fast geometric transformations of images for the purpose of data augmentation in deep learning. The supported geometric transformations are
* Translations
* Rigid transformations (translation + rotation)
* Similarity transformations (translation + rotation + isotropic scaling)
* Affine transformations (translation + rotation + arbitrary scaling + shearing)
* Deformable transformations (modeled as B-splines)

The tutorial below shows most functionality of this package. A small minimal working example is shown at the [end](#mwe).

To get started, we load the example image, and fashion it with some grid lines to show what each of the transformations do. The tutorial is using a 2D image, but in principle all code should generalize to arbitrary dimensions.

In [ ]:
image = plt.imread('lung_ct.png')
image = image[:, :, 0] # The image was loaded as RGB color image, but we only need one channel.
image[10::20] = 1
image[11::20] = 1
image[:, 10::20] = 1
image[:, 11::20] = 1

In [ ]:
plt.imshow(image)

## Transformations and interpolations, an example using translations

Spatial image transformations move the *sampling grid* of an image. This means that the effect of a transformation has the opposite effect on the image itself. For example, a translation to the *right* means that the *grid* is translated to the right, which means that the objects in you image move to the *left*. You can compare this to a camera: if you move the camera to the right, the objects on the picture will move to the left.

Let's make a translation transform that moves the image by 10% to the left:

In [ ]:
my_translation = gryds.TranslationTransformation([0, 0.1])

As you can see, the translation is defined in terms of *image indices* $i, j$, and not in terms of coordinates $x,y$. A translation of 10% to the left means that we have to move the sampling grid 10% to the right, which means we have to move the $j$-axis of the grid.

We can move points to check this. Here we move the point $(0.5, 0.5)$ which is the center of the grid:

In [ ]:
my_translation.transform([[0.5], [0.5]])

As you can see, the $j$-component has been shifted 10% to the right. When we apply the transformation to the image it can happen that the transformed grid points fall in between the points on the original grid. That is why we need to use an interpolator for this image. By default, we use a third-order B-spline interpolator.

**This is different from the B-spline we use to *deform* images later on!**

In [ ]:
my_image_interpolator = gryds.Interpolator(image)

We can transform the interpolator, to show the result:

In [ ]:
my_translated_image = my_image_interpolator.transform(my_translation)
plt.imshow(my_translated_image);

As you can see, the image has been shifted 10% to the left!

## Transformations via Grids

The `transform` method of the `Interpolator` object in the example above applies the translation to the image's grid. The following code is equivalent, but explicitly defines the Grid, transforms it, and uses the interpolator to resample on this grid:

In [ ]:
my_image_grid = gryds.Grid(image.shape)  # makes a Grid the size of the image
my_translated_image_grid = my_image_grid.transform(my_translation)
my_translated_image = my_image_interpolator.resample(my_translated_image_grid)

In [ ]:
plt.imshow(my_translated_image);

You can see what this transformation does to the grid by calculating the difference with the original grid:

In [ ]:
my_translated_image_grid.grid - my_image_grid.grid

## More advanced linear transformations

In a linear transformation, the transformation of points can be expressed as $\mathbf{x}_{new} = A \mathbf{x}_{old} + \mathbf{b}$, where $A$ is a matrix and $\mathbf{b}$ is a translation vector. Again note that the transformation has the opposite effect on the image itself.

We define linear transformations in a matrix $[A, \mathbf{b}]$, for example, a 45° rotation can be defined as

In [ ]:
angle = np.pi / 4.
my_rotation_matrix = [
    [np.cos(angle), -np.sin(angle), 0],
    [np.sin(angle), np.cos(angle), 0]
]

The transformation can be defined as

In [ ]:
my_rotation = gryds.LinearTransformation(my_rotation_matrix)

that we can now apply to the image:

In [ ]:
my_rotated_image = my_image_interpolator.transform(my_rotation)
plt.imshow(my_rotated_image);

Note that the center of rotation is the upper left corner (the $(0, 0)$ coordinate). We can change that by using the utility function `affine_matrix` that creates new linear matrices based on more meaningful parameters:

In [ ]:
my_rotation = gryds.AffineTransformation(
    ndim=2,
    angles=[np.pi/4.], # List of angles (for 3D transformations you need a list of 3 angles).
    center=[0.5, 0.5]  # Center of rotation.
)
my_rotated_image = my_image_interpolator.transform(my_rotation)
plt.imshow(my_rotated_image);

Notice that the image shows a clockwise rotation of the objects in the image, but that the definition of the transformation was counter clockwise. Again, this is because a counter clockwise rotation of the sampling grid results in a clockwise rotation of the objects in the image.

We can make more complicated transformations too, using more parameters, resulting in affine transformations.

In [ ]:
my_affine_transformation = gryds.AffineTransformation(
    ndim=2,
    angles=[np.pi/10.], # the rotation angle
    scaling=[1.2, 1.2], # the anisotropic scaling
    shear_matrix=[[1, 0.5], [0, 1]], # shearing matrix
    translation=[0., 0.], # translation
    center=[0.5, 0.5] # center of rotation
)
my_affine_image = my_image_interpolator.transform(my_affine_transformation)
plt.imshow(my_affine_image);

## Deformable transformations


Fully deformable transformations are great for biomedical applications, because it can transform biological tissue in a more or less realistic way.

Deformable transformations are implemented as B-spline transformations. They are defined as *displacements* on a grid. For two-dimensional images we implement two grids, one for the displacements in the $i$-directions, and one for the displacements in the $j$-directions, so for example, for a 3x3 grid:

In [ ]:
disp_i = np.array([
    [0.04, 0.02, 0.04],
    [-0.04, 0.01, -0.04],
    [-0.04, -0.01, -0.04]
])
disp_j = np.array([
    [0.04, -0.03, -0.05],
    [0.02, 0.03, 0.05],
    [0.02, -0.04, -0.05]
])

We then create a B-spline transformation

In [ ]:
my_bspline_transformation = gryds.BSplineTransformation([disp_i, disp_j])

and apply it to the image:

In [ ]:
my_deformed_image = my_image_interpolator.transform(my_bspline_transformation)
plt.imshow(my_deformed_image);

Beware of folding and tearing of the image. If you make the displacements too large, or the grid too fine, you will end up with unrealistic deformations, from which your network will likely be unable to learn anything useful.

In [ ]:
my_folding_bspline_transformation = gryds.BSplineTransformation([10*disp_i, 10*disp_j])
my_terribly_deformed_image = my_image_interpolator.transform(my_folding_bspline_transformation)
plt.imshow(my_terribly_deformed_image);

### Checking for folding using the Jacobian
You can inspect the folding by computing the Jacobian determinant on a `Grid` object. Wherever the Jacobian determinant is below zero, the image has folded, and information has disappeared.

In [ ]:
my_grid = gryds.Grid(image.shape)
jac_det = my_grid.jacobian_det(my_folding_bspline_transformation)

fig, ax = plt.subplots(1, 2)
ax[0].imshow(jac_det, vmin=-2e-5, vmax=5e-5);
ax[1].imshow(jac_det < 0);
ax[0].set_title('Jacobian determinant');
ax[1].set_title('Folding map');

For the well-behaved B-spline transformation we had before, the Jacobian does not show folding:

In [ ]:
my_grid = gryds.Grid(image.shape)
jac_det = my_grid.jacobian_det(my_bspline_transformation)

fig, ax = plt.subplots(1, 2)
ax[0].imshow(jac_det, vmin=-2e-5, vmax=5e-5);
ax[1].imshow(jac_det < 0);
ax[0].set_title('Jacobian determinant');
ax[1].set_title('Folding map');

### Plotting the displacement field
The displacement field can be plotted using a `Grid` object. The displacement field is the difference between the transformed grid and the original grid.

In [ ]:
my_grid = gryds.Grid(image.shape)
transformed_grid = my_grid.transform(my_folding_bspline_transformation)
displacement_field = transformed_grid.grid - my_grid.grid

fig, ax = plt.subplots(1, 2)
ax[0].imshow(**gryds.dfm_show(displacement_field[0]));
ax[1].imshow(**gryds.dfm_show(displacement_field[1]));
ax[0].set_title('$u_x$');
ax[1].set_title('$u_y$');

## Combining transformations

You can concatenate transformations (for example affine and a B-spline transformations) by simply putting a list of transformations to the `transform()` method of a interpolator. Note that the transformations are applied in reverse order. Again, this is because the transformations are applied to the sampling grid rather than the images. The example below first applies the 10% translation from above *first*, *then* the 45° rotation around the (0.5, 0.5) in the resulting image (!), and *then* the B-spline transformation. Even though multiple transformations are applied, the are all applied to the grid first before any interpolation in the image is performed.

In [ ]:
composed = gryds.ComposedTransformation(my_bspline_transformation, my_rotation, my_translation)
twice_transformed_image = my_image_interpolator.transform(composed)
plt.imshow(twice_transformed_image);

### Border modes when combining B-spline transformations

The BSplineTransformation has options for dealing with displacements at the border of the grid. These are similar to the interpolation options below. Options include 'constant', 'nearest', 'mirror', 'reflect', or 'wrap'. In most cases, only 'constant' and 'mirror' are useful. When using one transformation, these paramters do not change anything:

In [ ]:
bs_grid = 0.5 * (np.random.rand(2, 2, 2) - 0.5)

In [ ]:
my_bspline_transformation_constant = gryds.BSplineTransformation(bs_grid, mode='constant')
my_bspline_transformation_mirror = gryds.BSplineTransformation(bs_grid, mode='mirror')
my_deformed_image_constant = my_image_interpolator.transform(my_bspline_transformation_constant)
my_deformed_image_mirror = my_image_interpolator.transform(my_bspline_transformation_mirror)
fig, ax = plt.subplots(1, 3, figsize=(15, 5))
ax[0].imshow(my_deformed_image_constant); ax[0].set_title('Constant')
ax[1].imshow(my_deformed_image_mirror); ax[1].set_title('Mirror')
ax[2].imshow(my_deformed_image_constant - my_deformed_image_mirror); ax[2].set_title('Difference')

When combining transformations, there *is* a difference:

In [ ]:
my_affine_transformation = gryds.AffineTransformation(ndim=2, center=[0.5, 0.5], scaling=[1.5, 1.5])
my_deformed_image_constant = my_image_interpolator.transform(gryds.ComposedTransformation(my_affine_transformation, my_bspline_transformation_constant))
my_deformed_image_mirror = my_image_interpolator.transform(gryds.ComposedTransformation(my_affine_transformation, my_bspline_transformation_mirror))
fig, ax = plt.subplots(1, 3, figsize=(15, 5))
ax[0].imshow(my_deformed_image_constant); ax[0].set_title('Constant');
ax[1].imshow(my_deformed_image_mirror); ax[1].set_title('Mirror');
ax[2].imshow(my_deformed_image_constant - my_deformed_image_mirror); ax[2].set_title('Difference');

This is explained better when looking at the deformation fields, that clearly show that in the 'constant' case, the transformations are *not* combined outside the transformed 'frame' of the original image, which is particularly visible when using a scaling operation *after* a B-spline:

In [ ]:
grid = gryds.Grid((500, 500))
grid_t_constant = grid.transform(gryds.ComposedTransformation(my_affine_transformation, my_bspline_transformation_constant))
grid_t_mirror = grid.transform(gryds.ComposedTransformation(my_affine_transformation, my_bspline_transformation_mirror))

fig, ax = plt.subplots(1, 6, figsize=(15, 5))
ax[0].imshow(**gryds.dfm_show(grid_t_constant.grid[0] - grid.grid[0])); ax[0].set_title('$u_x$, constant');
ax[1].imshow(**gryds.dfm_show(grid_t_constant.grid[1] - grid.grid[1])); ax[1].set_title('$u_y$, constant');
ax[2].imshow(**gryds.dfm_show(grid_t_mirror.grid[0] - grid.grid[0])); ax[2].set_title('$u_x$, mirror');
ax[3].imshow(**gryds.dfm_show(grid_t_mirror.grid[1] - grid.grid[1])); ax[3].set_title('$u_y$, mirror');
ax[4].imshow(**gryds.dfm_show(grid_t_mirror.grid[0] - grid_t_constant.grid[0])); ax[4].set_title('$u_x$, diff');
ax[5].imshow(**gryds.dfm_show(grid_t_mirror.grid[1] - grid_t_constant.grid[1])); ax[5].set_title('$u_y$, diff');

Hence, if you combine multiple transformations, *do* use the 'mirror' mode.

# Interpolation options
The Interpolation object has parameters for which order of B-spline should be used, as well as how the interpolator should extrapolate. The options are
* `order`: The order of the B-spline used for interpolation.
* `mode`: Any of ‘constant’, ‘nearest’, 'mirror', ‘reflect’ or ‘wrap’.
* `cval`: The background value for `mode='constant'`

### Different background intensity

In [ ]:
my_image_interpolator = gryds.Interpolator(image, order=0, cval=1)
tr_image = my_image_interpolator.transform(my_rotation)
plt.imshow(tr_image);

### Border-modes

In [ ]:
fig, ax = plt.subplots(1, 5, figsize=(20, 5));
for i, mode in enumerate(['constant', 'nearest', 'reflect', 'mirror', 'wrap']):
    my_image_interpolator = gryds.Interpolator(image, mode=mode)
    tr_image = my_image_interpolator.transform(my_rotation)
    ax[i].imshow(tr_image);
    ax[i].set_title(mode);

### B-spline orders

In [ ]:
fig, ax = plt.subplots(1, 6, figsize=(20, 5));
for i in range(6):
    my_image_interpolator = gryds.Interpolator(image, order=i)
    tr_image = my_image_interpolator.transform(my_rotation)
    ax[i].imshow(tr_image);
    ax[i].set_title('Order {}'.format(i));

<a name="mwe"></a>
# A minimal working example for data augmentation

You can use the numpy.random module to make random transformations. During training of your network you can implement something like the code below:

In [ ]:
fig, ax = plt.subplots(5, 5, figsize=(15, 15));
ax = ax.flatten()
[x.set_axis_off() for x in ax];

for i in range(25):
    random_grid = np.random.rand(2, 3, 3) # Make a random 2D 3 x 3 grid
    random_grid -= 0.5 # Move the displacements to the -0.5 to 0.5 grid
    random_grid /= 5 # Scale the grid to -0.1 to 0.1 displacements

    my_augmentation = gryds.BSplineTransformation(random_grid)
    my_augmented_image = my_image_interpolator.transform(my_augmentation)

    ax[i].imshow(my_augmented_image);